# Random Forest - 최종
1. 목표
본 코드는 참외 농장의 생육 데이터에서 결측값을 처리하고, 시간별 평균을 계산하여 데이터를 정리하는 작업을 목표로 합니다.

2. 데이터 로드 및 평균 계산
주어진 엑셀 파일(growth_to_yield.xlsx)을 로드하여, MSRM_DT(측정일)를 기준으로 LEAF_LNGTH, LEAF_WDTH, STEM_THNS, SHPMN_QTY의 시간별 평균값을 계산하였습니다.
결측값을 최소화하고, 시간별 데이터의 평균적인 경향을 반영하기 위해 각 열에 대해 평균값을 계산한 후, MSRM_DT을 기준으로 그룹화하고 평균을 구하여 새로운 데이터프레임에 저장합니다.

3. 결측값 처리 (Advanced Interpolation)
결측값을 처리하기 위해 두 가지 방법을 사용합니다:
긴 결측 구간 처리 (길이가 5 이상인 구간):
NaN 값이 연속적으로 나타나는 긴 구간에 대해서는 앞뒤 채우기(forward-fill, backward-fill) 기법을 사용하여 결측값을 채웁니다.
단기 결측 구간 처리:
짧은 결측 구간에 대해서는 스플라인 보간법을 사용하여 결측값을 예측합니다. 스플라인 보간법은 데이터의 곡선적 특성을 반영하여 결측값을 보간할 수 있는 방법입니다.
결측값 처리 후, 임시로 추가된 열(NaN, NaN_Group)을 삭제하여 최종적인 데이터를 정리합니다.

4. 적용된 열
LEAF_LNGTH, LEAF_WDTH, STEM_THNS, SHPMN_QTY 4개의 열에 대해 결측값을 처리하는 방법을 적용했습니다.

5. 결과
시간별 평균값을 계산하고, 결측값을 효과적으로 보간하여 보다 정제된 데이터를 얻을 수 있었습니다. 이를 통해 추후 분석 및 예측 작업에 사용할 수 있는 정확한 데이터를 확보하였습니다.




*   스플라인 + 전후 보간



In [ ]:
import pandas as pd
import numpy as np

# 데이터 로드
file_path = '/content/growth_to_yield.xlsx'
df = pd.read_excel(file_path)

# 시간별 평균 계산 (INDVD_NO 제거)
df_avg = (
    df[['MSRM_DT', 'LEAF_LNGTH', 'LEAF_WDTH', 'STEM_THNS', 'SHPMN_QTY']]
    .groupby('MSRM_DT')
    .mean()  # 시간별 평균값 계산
    .reset_index()
)

# 특정 열에 대해 결측 구간 보완 (INNER_CBDX_CTRN처럼 여러 열에도 적용 가능)
def advanced_interpolation(df, column, threshold=5):
    # NaN 구간을 식별하고 각 NaN 구간의 길이를 계산
    df['NaN'] = df[column].isna().astype(int)
    df['NaN_Group'] = (df['NaN'].diff(1) != 0).cumsum() * df['NaN']
    nan_gap_lengths = df[df['NaN'] == 1].groupby('NaN_Group').size()

    # 긴 결측 구간 - 앞뒤 채우기
    long_gap_indices = nan_gap_lengths[nan_gap_lengths > threshold].index
    for idx in long_gap_indices:
        df.loc[df['NaN_Group'] == idx, column] = df[column].fillna(method='ffill').fillna(method='bfill')

    # 단기 결측 구간 - 스플라인 보간
    df[column] = df[column].interpolate(method='spline', order=3)


    # 임시 열 삭제
    df = df.drop(columns=['NaN', 'NaN_Group'])
    return df

# 열별로 적용 (예: LEAF_LNGTH, LEAF_WDTH 등)
for col in ['LEAF_LNGTH', 'LEAF_WDTH', 'STEM_THNS', 'SHPMN_QTY']:
    df_avg = advanced_interpolation(df_avg, col)


<ipython-input-41-82426ca37861>:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.loc[df['NaN_Group'] == idx, column] = df[column].fillna(method='ffill').fillna(method='bfill')
<ipython-input-41-82426ca37861>:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.loc[df['NaN_Group'] == idx, column] = df[column].fillna(method='ffill').fillna(method='bfill')
<ipython-input-41-82426ca37861>:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.loc[df['NaN_Group'] == idx, column] = df[column].fillna(method='ffill').fillna(method='bfill')
<ipython-input-41-82426ca37861>:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.loc[df['NaN_Group']



*   소숫점 둘째 자리까지만 나타나도록



In [ ]:
df_avg = df_avg.round(2)
df_avg

,MSRM_DT,LEAF_LNGTH,LEAF_WDTH,STEM_THNS,SHPMN_QTY
0,2019-01-01 00:00:00,10.42,10.80,3.58,340.0
1,2019-01-01 01:00:00,10.42,10.80,3.58,340.0
2,2019-01-01 02:00:00,10.42,10.80,3.58,340.0
3,2019-01-01 03:00:00,10.42,10.80,3.58,340.0
4,2019-01-01 04:00:00,10.42,10.80,3.58,340.0
...,...,...,...,...,...
5804,2019-08-30 20:00:00,11.60,10.87,5.57,120.0
5805,2019-08-30 21:00:00,11.60,10.87,5.57,120.0
5806,2019-08-30 22:00:00,11.60,10.87,5.57,120.0
5807,2019-08-30 23:00:00,11.60,10.87,5.57,120.0




*   보간된 최종 데이터셋 저장



In [ ]:
# 최종 dataset excel로 내보내기
df_avg.to_excel("growth_to_prod.xlsx")



#   Random Forest Regressor
1. 시간 피처 추가
데이터프레임에 MSRM_DT 열을 이용하여 hour, day, month 피처를 생성합니다. 이는 시간에 따른 패턴을 모델이 학습할 수 있도록 도와줍니다.
2. 독립 변수와 종속 변수 정의
X는 LEAF_LNGTH, LEAF_WDTH, STEM_THNS, hour, day, month로 구성된 독립 변수입니다.
y는 SHPMN_QTY로, 예측하고자 하는 종속 변수입니다.
3. 데이터 분할
train_test_split을 사용하여 데이터를 학습 데이터와 테스트 데이터로 나눕니다. 전체 데이터의 80%는 학습에, 20%는 테스트에 사용됩니다.
4. 모델 학습 및 예측
RandomForestRegressor 모델을 학습하고, 테스트 데이터에 대해 예측을 수행합니다.
5. 성능 평가
예측 결과에 대해 세 가지 평가 지표를 계산합니다:
평균 절대 오차 (MAE): 예측값과 실제값 간의 절대 차이의 평균입니다. MAE가 작을수록 모델의 예측이 실제값과 가까운 것입니다.
평균 제곱 오차 (MSE): 예측값과 실제값 간의 차이의 제곱 평균입니다. MSE는 오차가 클수록 더욱 큰 값을 가지며, 작은 MSE일수록 예측이 정확합니다.
결정 계수 (R²): 모델이 실제 데이터 변동성을 얼마나 설명하는지를 나타냅니다. 1에 가까울수록 모델이 데이터를 잘 설명한다고 볼 수 있습니다.
6. 출력
MAE, MSE, R² 값을 출력하여 모델 성능을 평가합니다.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 시간 피처 추가
df_avg['hour'] = pd.to_datetime(df_avg['MSRM_DT']).dt.hour
df_avg['day'] = pd.to_datetime(df_avg['MSRM_DT']).dt.day
df_avg['month'] = pd.to_datetime(df_avg['MSRM_DT']).dt.month

# 독립 변수와 종속 변수 정의
X = df_avg[['LEAF_LNGTH', 'LEAF_WDTH', 'STEM_THNS', 'hour', 'day', 'month']]
y = df_avg['SHPMN_QTY']

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Regressor 모델 학습
rf_model_time = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model_time.fit(X_train, y_train)

# 예측
rf_predictions_time = rf_model_time.predict(X_test)

# 성능 평가
rf_mae_time = mean_absolute_error(y_test, rf_predictions_time)
rf_mse_time = mean_squared_error(y_test, rf_predictions_time)
rf_r2_time = r2_score(y_test, rf_predictions_time)

print(f"평균 절대 오차 (MAE): {rf_mae_time}")
print(f"평균 제곱 오차 (MSE): {rf_mse_time}")
print(f"결정 계수 (R²): {rf_r2_time}")


평균 절대 오차 (MAE): 3.69948364888124
평균 제곱 오차 (MSE): 844.2829776247848
결정 계수 (R²): 0.9904357051572481
